# Putting together a database of MLB baseball game data

#### Dictionary of data fields for game log data

In [1]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [4]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [3]:
!rm mlb.db

In [5]:
game_log = pd.read_csv("game_log.csv", low_memory = False)
park_codes = pd.read_csv("park_codes.csv", low_memory = False)
person_codes = pd.read_csv("person_codes.csv", low_memory = False)
team_codes = pd.read_csv("team_codes.csv", low_memory = False)

In [6]:
# Game_logs
print('Total available baseball game logs:\n', game_log.shape)
game_log.head()

Total available baseball game logs:
 (171907, 161)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [7]:
game_log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

The "game_logs.csv" file is a collection of data from MLB baseball games played between 1871 and 2016.

There is data recorded for more than 170,000 games played in the given timespan.

The recorded data reflects a lot of information about the games played such as:

1.) Team wide statistics for home and visiting teams for each game played.

2.) Names of participating players for the home and visiting teams and their defensive positions.

3.) The name of the officiating umpire for each game.

4.) Designated "Winner" and "Loser" tags to the respective pitchers.


The only issues observed is the lack of a unique identifier to each game played which can serve as a "Primary Key" for the whole database.


In [8]:
print('Total No. of Baseball parks:\n', park_codes.shape[0])
park_codes.head()

Total No. of Baseball parks:
 252


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [9]:
park_codes.tail()

,park_id,name,aka,city,state,start,end,league,notes
247,WIL02,BB&T Ballpark at Bowman Field,NaN,Wiliamsport,PA,08/20/2017,08/20/2017,NL,PIT
248,WNY01,West New York Field Club Grounds,NaN,West New York,NJ,09/11/1898,09/17/1899,NL,"BRO:9/18&10/2/1898; NY1:9/11/98, 6/4&7/16&8/13&9/17/99"
249,WOR01,Agricultural County Fair Grounds I,NaN,Worcester,MA,05/01/1880,09/29/1882,NL,NaN
250,WOR02,Agricultural County Fair Grounds II,NaN,Worcester,MA,08/17/1887,08/17/1887,NL,1 BSN game
251,WOR03,Worcester Driving Park Grounds,NaN,Worcester,MA,10/30/1874,10/30/1874,NaN,1 BS1 game


The "park_codes" files is a collection of details about baseball stadiums across the USA. 

The 252 documented parks/stadiums are uniquely identified by their "park_id" which can serve as a key for the table, it can also be linked to "game_logs" data using it.

In [10]:
print('Total No. of Baseball players whose statistics are recorded:\n', person_codes.shape[0])
person_codes.head()

Total No. of Baseball players whose statistics are recorded:
 20494


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


In [11]:
person_codes.tail()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
20489,zuvep001,Zuvella,Paul,09/04/1982,NaN,04/02/1996,NaN
20490,zuveg101,Zuverink,George,04/21/1951,NaN,NaN,NaN
20491,zwild101,Zwilling,Dutch,08/14/1910,NaN,04/15/1941,NaN
20492,zycht001,Zych,Tony,09/04/2015,NaN,NaN,NaN
20493,thoma102,Thompson,NaN,NaN,NaN,NaN,NaN


The "person_codes" table is a collection of player data that documents when they made their debut in the MLB and under what capacity. Each player is uniquely identiifed by the "id" field.

For example, Paul Zuvella made his debut as a player in 1982. He went on to become a coach in the MLB in 1996. Similarly, some players transitioned from players to managerial and umpiring roles.

The distinction between "manager" and "coach" in baseball appears to mirror the world of soccer. The coach is tasked with the players intimately and improve their overall aptitute for the game whereas the manager is solely in charge of game tactics and is the face of the franchise to the media. 

In [12]:
print('Total No. of Baseball teams recorded:\n', team_codes.shape[0])
team_codes.head(10)

Total No. of Baseball teams recorded:
 150


,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1
5,BL2,AA,1882,1891,Baltimore,Orioles,BL2,1
6,BLN,NL,1892,1899,Baltimore,Orioles,BL2,2
7,BL4,NaN,1873,1873,Baltimore,Marylands,BL4,1
8,BLA,AL,1901,1902,Baltimore,Orioles,BLA,1
9,NYA,AL,1903,0,New York,Yankees,BLA,2


In [13]:
team_codes.tail(10)

,team_id,league,start,end,city,nickname,franch_id,seq
140,WS3,NaN,1871,1872,Washington,Olympics,WS3,1
141,WS4,NaN,1872,1872,Washington,Nationals,WS4,1
142,WS5,NaN,1873,1873,Washington,Nationals,WS5,1
143,WS6,NaN,1875,1875,Washington,Nationals,WS6,1
144,WS7,AA,1884,1884,Washington,Nationals,WS7,1
145,WS8,NL,1886,1889,Washington,Senators,WS8,1
146,WS9,AA,1891,1891,Washington,Senators,WS9,1
147,WSN,NL,1892,1899,Washington,Senators,WS9,2
148,WSU,UA,1884,1884,Washington,Nationals,WSU,1
149,MIA,NL,2012,0,Miami,Marlins,FLO,2


The "team_codes" table is a record of the teams professionally playing in the various leagues across USA.

The teams can identified by their "team_id". But, it appears that "team_id" seems to change based on the location of the franchise. For example, between 1901 and 1902, the Orioles played in Baltimore as part of the AL league. In 1903, they moved based from Baltimore to New York and became the Yankees. The "team_id" changed from "BLA" to "NYA".

In the above example, the only consistent between the transition from Baltimore Orioles to New York Yankees is "franch_id".

In [14]:
team_codes["franch_id"].value_counts().head(10)

BS1    4
BR3    3
MLA    3
SE1    3
LAA    3
TRN    3
PHA    3
CH2    2
FLO    2
HR1    2
Name: franch_id, dtype: int64

There appear to be 4 rows with the same "franch_id"

In [15]:
team_codes[team_codes["franch_id"] == "BS1"]

,team_id,league,start,end,city,nickname,franch_id,seq
21,BS1,NaN,1871,1875,Boston,Braves,BS1,1
22,BSN,NL,1876,1952,Boston,Braves,BS1,2
23,MLN,NL,1953,1965,Milwaukee,Braves,BS1,3
24,ATL,NL,1966,0,Atlanta,Braves,BS1,4


"franch_id" refers to franchise id, it is highly useful to tracks the movement of a franchise across multiple cities. For example, the Atlanta Braves started out as the Boston Braves, transitioning to the Milwaukee Braves before settling in Atlanta.

### Postions in Baseball

In [16]:
game_log["v_player_1_def_pos"].value_counts()

8.0     38715
4.0     28458
6.0     22220
7.0     19595
9.0     14965
5.0     11923
3.0      3383
10.0     1062
2.0       515
1.0         2
Name: v_player_1_def_pos, dtype: int64

Baseball could be considered a complicated sport to get accustomed to and follow for the new and casual fan.
To address that, let us briefly get acquainted with the various positions in the sport.

There are 9 positions on the field in baseball, [this link is a detailed explanation of each position and their functions.](http://baseballcoachinglab.com/baseball-positions)

| Position number | Position Name |
| ----------- | ----------- |
| 1   | Pitcher  |
| 2   | Catcher  |
| 3   | First Baseman  |
| 4   | Second Baseman  |
| 5   | Third Baseman  |
| 6   | Shortstop  |
| 7   | Left Fielder  |
| 8   | Center Fielder  |
| 9   | Right Fielder  |

The postion marked 10 could possibly refer to the pinch/designated hitter. They are batters who are subbed into the batting lineup for the pitcher.

### Leagues in Baseball

In [17]:
game_log["v_league"].value_counts(dropna=False)
#game_log["v_league"].unique()

NL     88866
AL     74713
AA      5039
FL      1243
NaN     1086
PL       532
UA       428
Name: v_league, dtype: int64

Modern day MLB is split into 2 leagues namely AL (American League) and NL (National League). An overwhelming majority of the games recorded in the "game_log" belong to teams from either of these 2 leagues.

Outside the 1086 "NaN" designations, we have ~7000 games played my teams belonging to leagues outside NL and AL. It would be worth investigating the years of operation of the various leagues to see evidence of overlapping timeframes.

In [18]:
def functioning_years(league_info):
    temp_league = game_log[game_log["v_league"] == league_info]
    start_date = temp_league["date"].min()
    end_date = temp_league["date"].max()
    return ((start_date, end_date))
    
mlb_leagues = game_log["v_league"].unique()
mlb_league_timeline = []
for each in mlb_leagues:
    temp = functioning_years(each)
    mlb_league_timeline.append([each, temp[0], temp[1]])

mlb_league_timeline = pd.DataFrame(mlb_league_timeline,columns=['League', 'Start_date', 'End_date'])
mlb_league_timeline = mlb_league_timeline.drop(mlb_league_timeline.index[0])
mlb_league_timelines = mlb_league_timeline.sort_values(by=["End_date"])
mlb_league_timelines

,League,Start_date,End_date
3,UA,18840417.0,18841019.0
4,PL,18900419.0,18901004.0
2,AA,18820502.0,18911006.0
6,FL,19140413.0,19151003.0
1,NL,18760422.0,20161002.0
5,AL,19010424.0,20161002.0


Based off the dates the leagues ceased to exist, we can see that the NL and AL are obviously the longest running leagues upto the last date of recorded data.

Additional detailed information about the non-existent leagues can be found at the links provided.

UA: [Union Association](https://en.wikipedia.org/wiki/Union_Association)
PL: [Players League](https://en.wikipedia.org/wiki/Players%27_League)
AA: [American Association](https://en.wikipedia.org/wiki/American_Association_%2819th_century%29)
FL: [Federal League](https://en.wikipedia.org/wiki/Federal_League)



## Transforming dataframesinto SQL tables

In [19]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(q):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        return conn.execute(q)
    
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [20]:
tables_dict = {
    "game_log":game_log,
    "team_codes":team_codes,
    "park_codes":park_codes,
    "person_codes":person_codes
}

with sqlite3.connect("mlb.db") as conn:
    for key,value in tables_dict.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(key))
        value.to_sql(key, conn, index=False)

In [21]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table


In [22]:
c1 = """
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
"""

run_command(c1)

In [23]:
c2 = """
UPDATE game_log
SET game_id = date || h_name || number_of_game
WHERE game_id IS NULL; 
"""

run_command(c2)

In [24]:
query1 = """
SELECT
    game_id,
    date,
    h_name,
    number_of_game
FROM game_log
LIMIT 5;
"""

run_query(query1)

,game_id,date,h_name,number_of_game
0,18710504FW10,18710504,FW1,0
1,18710505WS30,18710505,WS3,0
2,18710506RC10,18710506,RC1,0
3,18710508CH10,18710508,CH1,0
4,18710509TRO0,18710509,TRO,0


## Normalizing the data to remove redundancy

The following are opportunities for normalization of our data:

* In person_codes, the debut dates could be reproduced using game log data, the earliest appearance of a player in a game will correspond to their debut.  
* Similar treatment can be done to the start and end dates of teams in team_codes and parks in park_codes, based off their first and last appearance in game_log.  
* Player names in game_log can be removed and instead be linked with the person_codes table using player_id.  
* In game_log, the game stats are repeated for the home and visting teams. These could be separated into a different table to reduce redundancy.    
* The game_log table has records for all 9 players for the home and visiting teams and their positions which equates to 54 columns in total. Instead, we can have a separate table recording player appearances and the positions they played in. 
* We can do a similar thing with the umpires from game_log, instead of listing all four positions as columns, we can put the umpires either in their own table.

### Proposed schema design for MLB database

![A proposed schema for easier access to the MLB game logs](https://imgur.com/db0pMBu.jpg)


## Creating necessary tables

In [25]:
c1 = """
CREATE TABLE IF NOT EXISTS player_details (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
"""

c2 = """
INSERT OR IGNORE INTO player_details
SELECT
    id,
    first,
    last
FROM person_codes;
"""

q = """
SELECT * FROM player_details
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [26]:
c1 = """
CREATE TABLE IF NOT EXISTS park_locations (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    park_nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
"""

c2 = """
INSERT OR IGNORE INTO park_locations
SELECT
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
"""

q = """
SELECT * FROM park_locations
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,park_id,name,park_nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [27]:
c1 = "DROP TABLE IF EXISTS appearance_type;"

run_command(c1)

c2 = """
CREATE TABLE IF NOT EXISTS appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(c2)

appearance_type = pd.read_csv("appearance_type.csv", low_memory=False)

with sqlite3.connect("mlb.db") as conn:
    appearance_type.to_sql('appearance_type', conn, index=False, if_exists="append")
        

q = """
SELECT * FROM appearance_type
LIMIT 15;
"""

run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [28]:
c1 = """
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
);
"""

c2 = """
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association")
;
"""

q = """
SELECT * FROM league
"""

run_command(c1)
run_command(c2)
run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


In [29]:
c1 = """
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    team_name TEXT,
    city TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id) 
);
"""

c2 = """
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
"""

q = """
SELECT * FROM team
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,team_name,city,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [30]:
c1 = """
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    legnth_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park_locations(park_id)
);
"""

c2 = """
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
"""

q = """
SELECT * FROM game
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,legnth_minutes,additional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,18710505WS30,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,18710508CH10,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,18710509TRO0,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


In [31]:
c1 = """
CREATE TABLE IF NOT EXISTS team_statistics (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id)
);
"""

run_command(c1)

In [32]:
c2 = """
INSERT OR IGNORE INTO team_statistics
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""

run_command(c2)

In [33]:
q = """
SELECT * FROM team_statistics
ORDER By game_id, home
LIMIT 10;
"""

run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,CL1,18710504FW10,0,None,0,000000000,30,4,1,0,0,0,0,0,0,1,None,6,1,None,-1,None,4,1,1,1,0,0,27,9,0,3,0,0
1,FW1,18710504FW10,1,None,2,010010000,31,4,1,0,0,2,0,0,0,1,None,0,0,None,-1,None,3,1,0,0,0,0,27,3,3,1,1,0
2,BS1,18710505WS30,0,None,20,107000435,41,13,1,2,0,13,0,0,0,18,None,5,3,None,-1,None,12,1,6,6,1,0,27,13,10,1,2,0
3,WS3,18710505WS30,1,None,18,640113030,49,14,2,0,0,11,0,0,0,10,None,2,1,None,-1,None,14,1,7,7,0,0,27,20,10,2,3,0
4,CL1,18710506RC10,0,None,12,610020003,49,11,1,1,0,8,0,0,0,0,None,1,0,None,-1,None,10,1,0,0,2,0,27,12,8,5,0,0
5,RC1,18710506RC10,1,None,4,010020100,36,7,2,1,0,2,0,0,0,0,None,3,5,None,-1,None,5,1,3,3,1,0,27,12,13,3,0,0
6,CL1,18710508CH10,0,None,12,101403111,46,15,2,1,2,10,0,0,0,0,None,1,0,None,-1,None,7,1,6,6,0,0,27,15,11,6,0,0
7,CH1,18710508CH10,1,None,14,077000000,43,11,2,0,0,8,0,0,0,4,None,2,1,None,-1,None,6,1,4,4,0,0,27,14,7,2,0,0
8,BS1,18710509TRO0,0,None,9,000002232,46,17,4,1,0,6,0,0,0,2,None,0,1,None,-1,None,12,1,2,2,0,0,27,12,5,0,1,0
9,TRO,18710509TRO0,1,None,5,101003000,36,9,0,0,0,2,0,0,0,3,None,0,2,None,-1,None,7,1,3,3,1,0,27,11,7,3,0,0


In [34]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES player_details(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""
run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))

In [35]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646109,porta901,None,20161002WAS0,U1B,First Base,umpire
1,1646108,onorb901,None,20161002WAS0,U2B,Second Base,umpire
2,1646107,kellj901,None,20161002WAS0,U3B,Third Base,umpire
3,1646110,tumpj901,None,20161002WAS0,UHP,Home Plate,umpire
4,1646111,brica001,MIA,20161002WAS0,ALP,Losing Pitcher,award
5,6716279,koeht001,MIA,20161002WAS0,D1,Pitcher,defense
6,4744553,telit001,MIA,20161002WAS0,D2,Catcher,defense
7,5589581,bourj002,MIA,20161002WAS0,D3,1st Base,defense
8,4462877,gordd002,MIA,20161002WAS0,D4,2nd Base,defense
9,5026229,pradm001,MIA,20161002WAS0,D5,3rd Base,defense


## Removing original tables from database

In [36]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,player_details,table
5,park_locations,table
6,appearance_type,table
7,league,table
8,team,table
9,game,table


In [37]:
tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in tables:
    c = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c)

show_tables()

,name,type
0,player_details,table
1,park_locations,table
2,appearance_type,table
3,league,table
4,team,table
5,game,table
6,team_statistics,table
7,person_appearance,table
